In [1]:
import pyodbc
import json
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import asPolygon, box, LineString, Point, Polygon
from geopy.distance import geodesic
from time import time
from keplergl import KeplerGl
import pickle
import difflib
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings

<function warnings.filterwarnings(action, message='', category=<class 'Warning'>, module='', lineno=0, append=False)>

In [3]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [4]:
gpd_regions = gpd.read_file('D:\\work flow\\2020\\Feb\\Region\\REVISED_MAPPED_REGIONS_ONLY_FEB_2019-polygon.shp').to_crs(epsg=4326)
gpd_Sales_Area = gpd.read_file('D:\\work flow\\2020\\Feb\\Sales Area1\\REVISED_MAPPED_SALES_AREAS_JULY_LATEST-polygon.shp').to_crs(epsg=4326)
gpd_TDR = gpd.read_file('D:\\work flow\\2020\Feb\\TDR\\TDR_CLUSTERS_NOV_2019_WORKING-polygon.shp').to_crs(epsg=4326)

In [5]:
gpd_regions.head()

,REGION,Area,Pop,AvgFloat,Stores,geometry
0,GREATER WESTERN,23486.771088,0.0,6.635600e+08,28657.0,"MULTIPOLYGON (((34.13547 -0.47056, 34.13721 -0..."
1,RIFT,152085.704549,0.0,8.297802e+08,30626.0,"MULTIPOLYGON (((36.60725 2.36543, 36.60859 2.3..."
2,NAIROBI WEST,148157.763631,0.0,7.448693e+09,215257.0,"POLYGON ((36.74132 -1.32402, 36.73833 -1.32276..."
3,MT.KENYA,429980.436181,0.0,8.226809e+09,238861.0,"POLYGON ((40.22253 -1.98496, 40.21214 -1.96226..."
4,NAIROBI EAST,98253.556295,0.0,4.622628e+09,134431.0,"MULTIPOLYGON (((36.80239 -1.31840, 36.80314 -1..."


gpd_regions=gpd_regions.rename(columns={'geometry':'Region_Polygon'})
gpd_Sales_Area=gpd_Sales_Area.rename(columns={'geometry':'Sales_polygon'})
merged = gpd_regions.merge(gpd_Sales_Area, on='REGION')
merged=merged[['REGION','SALESAREA','SUBREGION','Region_Polygon','Sales_polygon']]

In [6]:
Region_map = KeplerGl(height=800)
Sales_map = KeplerGl(height=800)
TDR_map = KeplerGl(height=800)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md
User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md
User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


In [7]:
Region_map.add_data(data=gpd_regions, name="Saf_Regions")
Region_map

KeplerGl(data={'Saf_Regions': {'index': [0, 1, 2, 3, 4, 5], 'columns': ['REGION', 'Area', 'Pop', 'AvgFloat', '…

In [8]:
Sales_map.add_data(data=gpd_Sales_Area, name="Saf_Sales_Areas")
Sales_map

KeplerGl(data={'Saf_Sales_Areas': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,…

In [9]:
TDR_map.add_data(data=gpd_TDR, name="Saf_TDR_Areas")
TDR_map

KeplerGl(data={'Saf_TDR_Areas': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 1…

In [10]:
null_integer = -9999999999  # with 10 9s
    
Conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                             'Server=10.184.38.208;'
                             'Database=CEM_KPIs;'
                             'UID=powerbi;'
                             'PWD=Powerbi@1;')

query = (
'SELECT [LAC],[Cell_Name],[Node_Name],[Region],[ID],[Vendor],[TECHNOLOGY],[LATITUDE_DEC],[LONGITUDE_DEC],[Site_Name]'
',[BAND],[Azimuth],[distname_cell_name],[Sales_Area],[Sales_Zone]'
',[Field_Engineer],[Optimization_Engineer],[Cell_Classification],[RAN_did],a1.*FROM [CEM_KPIs].[kpi].[Dashboard_KPIs_NEI]'
"as a1 inner join [ID_Mapping_Dr].[dbo].[CGI_MAP] as a2 on a1.did=a2.id where Date>='2020-02-10' "
)

In [11]:
def setting_cells_df_datatypes(cells_df):
    cells_df.Date = pd.to_datetime(cells_df.Date)

    cells_df.Azimuth = cells_df.Azimuth.astype(float)
    
    # Removing the special characters from the name Columns
    # Repeating it might be faster than using a loop TODO: Confirm if this is true
    cells_df['Cell_Name'] = cells_df['Cell_Name'].str.translate({ord(c): " " for c in "!@#$%^&*()[]{};:,./<>?\|`~=+"})
    cells_df['Site_Name'] = cells_df['Site_Name'].str.translate({ord(c): " " for c in "!@#$%^&*()[]{};:,./<>?\|`~=+"})
    cells_df['Region']=cells_df['Region'].replace({'MT. KENYA':'MT.KENYA'})
    cells_df['Date']=cells_df['Date'].astype(str)
    

    return cells_df

In [12]:
cells_df = setting_cells_df_datatypes(pd.read_sql(query, Conn))

In [13]:
cells_df=cells_df.head(5000)

In [14]:
#gpd_regions=gpd_regions.rename(columns={'REGION':'Region'})
#Regional=pd.merge(cells_df,gpd_regions, on=['Region'])
#Regional.shape

In [15]:
def difference(df1, df2, which=None):
    """Find rows which are different between two DataFrames."""
    comparison_df = df1.merge(df2,
                              indicator=True,
                              how='outer')
    if which is None:
        diff_df = comparison_df[comparison_df['_merge'] != 'both']
    else:
        diff_df = comparison_df[comparison_df['_merge'] == which]
    #diff_df.to_csv('data/diff.csv')
    return diff_df

In [16]:
#Filling nulls
'''columns=['Data_SR_BH','CEM_CSSR','CEM_CDR','setup_time','TCH_Availability_Daily']
for col in columns:
    cells_df[col]=Regional[col].fillna(null_integer)'''

"columns=['Data_SR_BH','CEM_CSSR','CEM_CDR','setup_time','TCH_Availability_Daily']\nfor col in columns:\n    cells_df[col]=Regional[col].fillna(null_integer)"

In [17]:
#Define a function to calculate CSSR NEI
def NEI(data):
    for i, row in data.iterrows():
        if data['CEM_CSSR'][i]>=99.5:
            data['NEI CSSR']=100
        elif data['CEM_CSSR'][i]>=99:
            data['NEI CSSR']=80 + (data['CEM_CSSR']-99)*(100-80)/(99.5-99)
        elif data['CEM_CSSR'][i]>=85:
            data['NEI CSSR']=0 +(data['CEM_CSSR']-85)*(80-0)/(99-85)
        else:
            data['NEI CSSR']=0
    return(data)

In [18]:
NEI(cells_df)

,LAC,Cell_Name,Node_Name,Region,ID,Vendor,TECHNOLOGY,LATITUDE_DEC,LONGITUDE_DEC,Site_Name,...,CS_RRC_Success_Daily_N1,CS_RRC_Attempts_Daily_D1,CS_RAB_Success_Daily_N2,CS_RAB_Attempts_Daily_D2,Data_SR_Daily,Data_SR_BH,CEM_CSSR,CEM_CDR,setup_time,NEI CSSR
0,639-02-4526,13865_NE_NI2370-Kibera_South_OUTE-0,ATHRNC1501,NAIROBI EAST,4024,Huawei,3G,-1.318720,36.783600,13865_NE_NI2370-Kibera_South_OUTE_MGF,...,1467.0,1471.0,1847.0,1847.0,99.333153,100.000000,99.618324,0.063857,0.612605,104.732971
1,639-02-4526,13865_NE_NI2370-Kibera_South_OUTE-1,ATHRNC1501,NAIROBI EAST,4026,Huawei,3G,-1.318720,36.783600,13865_NE_NI2370-Kibera_South_OUTE_MGF,...,NaN,NaN,NaN,NaN,98.729019,98.722740,98.604500,0.582751,2.834374,64.179993
2,639-02-2518,13814_MK_EC2370-Meru_Tuntu_OUTA-2,NYRRNC4004,MT.KENYA,4027,Huawei,3G,0.054487,37.642799,13814_MK_EC2370-Meru_Tuntu_OUTA_MGF,...,8102.0,8104.0,7772.0,7773.0,99.939789,100.000000,99.190575,0.114531,4.352467,87.622986
3,639-02-4510,13605_NE_NI2379-Kayole_Section_1_OUTE-5,QOARNC1100,NAIROBI EAST,4111,Huawei,3G,-1.265400,36.917500,13605_NE_NI2379-Kayole_Section_1_OUTE_MRT,...,NaN,NaN,NaN,NaN,99.587761,99.388428,99.054306,0.246063,2.737468,82.172241
4,639-02-4521,13760_NE_NI2385-Nacico_OUTE_MWM-6,SCCRNC1001,NAIROBI EAST,4175,Huawei,3G,-1.287060,36.837200,13760_NE_NI2385-Nacico_OUTE_MWM,...,NaN,NaN,NaN,NaN,99.303093,98.305084,99.183762,0.232116,5.503750,87.350464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,639-02-5506,14672_NW_NM5425-Githurai_Roundabout_OUTH_U900-1,QOARNC1106,NAIROBI WEST,32959,Huawei,3G,-1.203720,36.918900,14672_NW_NM3425-Githurai_Roundabout_OUTH_MGF,...,3481.0,3488.0,4211.0,4227.0,95.807549,97.935661,98.462044,0.166168,4.464143,58.481750
4996,639-02-4523,14698_NW_NM5426-Zimmerman_Njathaini_OUTH_U900-1,QOARNC0512,NAIROBI WEST,32968,Huawei,3G,-1.204560,36.896198,14698_NW_NM3426-Zimmerman_Njathaini_OUTH_MGF,...,NaN,NaN,NaN,NaN,98.173683,96.613144,98.824318,0.271924,2.154833,72.972717
4997,639-02-5571,14223_WN_WL5427-Koru_MGF_UMTS900-4,CHRRNC2501,GREATER WESTERN,32986,Nokia,3G,-0.181306,35.269699,14223_WN_WL3427-Koru_MGF,...,NaN,NaN,NaN,NaN,99.009644,98.796722,99.114143,0.263736,2.188611,84.565735
4998,639-02-2530,15582_CO_EL3507-Dabasso_Dongo_Kundu_MGF-0,MSARNC3004,COAST,12874,Nokia,3G,-3.379890,39.980099,15582_CO_EL3507-Dabasso_Dongo_Kundu_MGF,...,NaN,NaN,NaN,NaN,98.384926,98.235291,99.451752,0.053349,2.210723,98.070068


In [19]:
#Define a function to calculate Data SR NEI
def NEI(data):
    for i, row in data.iterrows():
        if data['Data_SR_BH'][i]>=99.5:
            data['Data SR NEI']=100
        elif data['Data_SR_BH'][i]>=99:
            data['Data SR NEI']=80 + (data['Data_SR_BH']-99)*(100-80)/(99.5-99)
        elif data['Data_SR_BH'][i]>=85:
            data['Data SR NEI']=0 +(data['Data_SR_BH']-85)*(80-0)/(99-85)
        else:
            data['Data SR NEI']=0
    return(data)

In [20]:
NEI(cells_df)

,LAC,Cell_Name,Node_Name,Region,ID,Vendor,TECHNOLOGY,LATITUDE_DEC,LONGITUDE_DEC,Site_Name,...,CS_RRC_Attempts_Daily_D1,CS_RAB_Success_Daily_N2,CS_RAB_Attempts_Daily_D2,Data_SR_Daily,Data_SR_BH,CEM_CSSR,CEM_CDR,setup_time,NEI CSSR,Data SR NEI
0,639-02-4526,13865_NE_NI2370-Kibera_South_OUTE-0,ATHRNC1501,NAIROBI EAST,4024,Huawei,3G,-1.318720,36.783600,13865_NE_NI2370-Kibera_South_OUTE_MGF,...,1471.0,1847.0,1847.0,99.333153,100.000000,99.618324,0.063857,0.612605,104.732971,100
1,639-02-4526,13865_NE_NI2370-Kibera_South_OUTE-1,ATHRNC1501,NAIROBI EAST,4026,Huawei,3G,-1.318720,36.783600,13865_NE_NI2370-Kibera_South_OUTE_MGF,...,NaN,NaN,NaN,98.729019,98.722740,98.604500,0.582751,2.834374,64.179993,100
2,639-02-2518,13814_MK_EC2370-Meru_Tuntu_OUTA-2,NYRRNC4004,MT.KENYA,4027,Huawei,3G,0.054487,37.642799,13814_MK_EC2370-Meru_Tuntu_OUTA_MGF,...,8104.0,7772.0,7773.0,99.939789,100.000000,99.190575,0.114531,4.352467,87.622986,100
3,639-02-4510,13605_NE_NI2379-Kayole_Section_1_OUTE-5,QOARNC1100,NAIROBI EAST,4111,Huawei,3G,-1.265400,36.917500,13605_NE_NI2379-Kayole_Section_1_OUTE_MRT,...,NaN,NaN,NaN,99.587761,99.388428,99.054306,0.246063,2.737468,82.172241,100
4,639-02-4521,13760_NE_NI2385-Nacico_OUTE_MWM-6,SCCRNC1001,NAIROBI EAST,4175,Huawei,3G,-1.287060,36.837200,13760_NE_NI2385-Nacico_OUTE_MWM,...,NaN,NaN,NaN,99.303093,98.305084,99.183762,0.232116,5.503750,87.350464,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,639-02-5506,14672_NW_NM5425-Githurai_Roundabout_OUTH_U900-1,QOARNC1106,NAIROBI WEST,32959,Huawei,3G,-1.203720,36.918900,14672_NW_NM3425-Githurai_Roundabout_OUTH_MGF,...,3488.0,4211.0,4227.0,95.807549,97.935661,98.462044,0.166168,4.464143,58.481750,100
4996,639-02-4523,14698_NW_NM5426-Zimmerman_Njathaini_OUTH_U900-1,QOARNC0512,NAIROBI WEST,32968,Huawei,3G,-1.204560,36.896198,14698_NW_NM3426-Zimmerman_Njathaini_OUTH_MGF,...,NaN,NaN,NaN,98.173683,96.613144,98.824318,0.271924,2.154833,72.972717,100
4997,639-02-5571,14223_WN_WL5427-Koru_MGF_UMTS900-4,CHRRNC2501,GREATER WESTERN,32986,Nokia,3G,-0.181306,35.269699,14223_WN_WL3427-Koru_MGF,...,NaN,NaN,NaN,99.009644,98.796722,99.114143,0.263736,2.188611,84.565735,100
4998,639-02-2530,15582_CO_EL3507-Dabasso_Dongo_Kundu_MGF-0,MSARNC3004,COAST,12874,Nokia,3G,-3.379890,39.980099,15582_CO_EL3507-Dabasso_Dongo_Kundu_MGF,...,NaN,NaN,NaN,98.384926,98.235291,99.451752,0.053349,2.210723,98.070068,100


In [21]:
#Define a function to calculate CDR NEI
def NEI(data):
    for i, row in data.iterrows():
        if data['CEM_CDR'][i]<=0.25:
            data['CDR NEI']=100
        elif data['CEM_CDR'][i]<=0.3:
            data['CDR NEI']=80 + (0.3-data['CEM_CDR'])*(100-80)/(0.3-0.25)
        elif data['CEM_CDR'][i]<=0.5:
            data['CDR NEI']=0 +(0.5-data['CEM_CDR'])*(80-0)/(0.5-0.3)
        else:
            data['CDR NEI']=0
    return(data)

In [22]:
NEI(cells_df)

,LAC,Cell_Name,Node_Name,Region,ID,Vendor,TECHNOLOGY,LATITUDE_DEC,LONGITUDE_DEC,Site_Name,...,CS_RAB_Success_Daily_N2,CS_RAB_Attempts_Daily_D2,Data_SR_Daily,Data_SR_BH,CEM_CSSR,CEM_CDR,setup_time,NEI CSSR,Data SR NEI,CDR NEI
0,639-02-4526,13865_NE_NI2370-Kibera_South_OUTE-0,ATHRNC1501,NAIROBI EAST,4024,Huawei,3G,-1.318720,36.783600,13865_NE_NI2370-Kibera_South_OUTE_MGF,...,1847.0,1847.0,99.333153,100.000000,99.618324,0.063857,0.612605,104.732971,100,100
1,639-02-4526,13865_NE_NI2370-Kibera_South_OUTE-1,ATHRNC1501,NAIROBI EAST,4026,Huawei,3G,-1.318720,36.783600,13865_NE_NI2370-Kibera_South_OUTE_MGF,...,NaN,NaN,98.729019,98.722740,98.604500,0.582751,2.834374,64.179993,100,100
2,639-02-2518,13814_MK_EC2370-Meru_Tuntu_OUTA-2,NYRRNC4004,MT.KENYA,4027,Huawei,3G,0.054487,37.642799,13814_MK_EC2370-Meru_Tuntu_OUTA_MGF,...,7772.0,7773.0,99.939789,100.000000,99.190575,0.114531,4.352467,87.622986,100,100
3,639-02-4510,13605_NE_NI2379-Kayole_Section_1_OUTE-5,QOARNC1100,NAIROBI EAST,4111,Huawei,3G,-1.265400,36.917500,13605_NE_NI2379-Kayole_Section_1_OUTE_MRT,...,NaN,NaN,99.587761,99.388428,99.054306,0.246063,2.737468,82.172241,100,100
4,639-02-4521,13760_NE_NI2385-Nacico_OUTE_MWM-6,SCCRNC1001,NAIROBI EAST,4175,Huawei,3G,-1.287060,36.837200,13760_NE_NI2385-Nacico_OUTE_MWM,...,NaN,NaN,99.303093,98.305084,99.183762,0.232116,5.503750,87.350464,100,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,639-02-5506,14672_NW_NM5425-Githurai_Roundabout_OUTH_U900-1,QOARNC1106,NAIROBI WEST,32959,Huawei,3G,-1.203720,36.918900,14672_NW_NM3425-Githurai_Roundabout_OUTH_MGF,...,4211.0,4227.0,95.807549,97.935661,98.462044,0.166168,4.464143,58.481750,100,100
4996,639-02-4523,14698_NW_NM5426-Zimmerman_Njathaini_OUTH_U900-1,QOARNC0512,NAIROBI WEST,32968,Huawei,3G,-1.204560,36.896198,14698_NW_NM3426-Zimmerman_Njathaini_OUTH_MGF,...,NaN,NaN,98.173683,96.613144,98.824318,0.271924,2.154833,72.972717,100,100
4997,639-02-5571,14223_WN_WL5427-Koru_MGF_UMTS900-4,CHRRNC2501,GREATER WESTERN,32986,Nokia,3G,-0.181306,35.269699,14223_WN_WL3427-Koru_MGF,...,NaN,NaN,99.009644,98.796722,99.114143,0.263736,2.188611,84.565735,100,100
4998,639-02-2530,15582_CO_EL3507-Dabasso_Dongo_Kundu_MGF-0,MSARNC3004,COAST,12874,Nokia,3G,-3.379890,39.980099,15582_CO_EL3507-Dabasso_Dongo_Kundu_MGF,...,NaN,NaN,98.384926,98.235291,99.451752,0.053349,2.210723,98.070068,100,100


In [23]:
#Define a function to calculate Availability NEI
def NEI(data):
    for i, row in data.iterrows():
        if data['TCH_Availability_Daily'][i]>=99.85:
            data['Availability NEI']=80 + ((data['TCH_Availability_Daily']-99.85)*(100-80)/(100-99.85))
        elif data['TCH_Availability_Daily'][i]>=90:
            data['Availability NEI']=0 +((data['TCH_Availability_Daily']-90)*(80-0)/(99.85-90))
        else:
            data['Availability NEI']=0
    return(data)

In [24]:
NEI(cells_df)

,LAC,Cell_Name,Node_Name,Region,ID,Vendor,TECHNOLOGY,LATITUDE_DEC,LONGITUDE_DEC,Site_Name,...,CS_RAB_Attempts_Daily_D2,Data_SR_Daily,Data_SR_BH,CEM_CSSR,CEM_CDR,setup_time,NEI CSSR,Data SR NEI,CDR NEI,Availability NEI
0,639-02-4526,13865_NE_NI2370-Kibera_South_OUTE-0,ATHRNC1501,NAIROBI EAST,4024,Huawei,3G,-1.318720,36.783600,13865_NE_NI2370-Kibera_South_OUTE_MGF,...,1847.0,99.333153,100.000000,99.618324,0.063857,0.612605,104.732971,100,100,100.0
1,639-02-4526,13865_NE_NI2370-Kibera_South_OUTE-1,ATHRNC1501,NAIROBI EAST,4026,Huawei,3G,-1.318720,36.783600,13865_NE_NI2370-Kibera_South_OUTE_MGF,...,NaN,98.729019,98.722740,98.604500,0.582751,2.834374,64.179993,100,100,100.0
2,639-02-2518,13814_MK_EC2370-Meru_Tuntu_OUTA-2,NYRRNC4004,MT.KENYA,4027,Huawei,3G,0.054487,37.642799,13814_MK_EC2370-Meru_Tuntu_OUTA_MGF,...,7773.0,99.939789,100.000000,99.190575,0.114531,4.352467,87.622986,100,100,100.0
3,639-02-4510,13605_NE_NI2379-Kayole_Section_1_OUTE-5,QOARNC1100,NAIROBI EAST,4111,Huawei,3G,-1.265400,36.917500,13605_NE_NI2379-Kayole_Section_1_OUTE_MRT,...,NaN,99.587761,99.388428,99.054306,0.246063,2.737468,82.172241,100,100,100.0
4,639-02-4521,13760_NE_NI2385-Nacico_OUTE_MWM-6,SCCRNC1001,NAIROBI EAST,4175,Huawei,3G,-1.287060,36.837200,13760_NE_NI2385-Nacico_OUTE_MWM,...,NaN,99.303093,98.305084,99.183762,0.232116,5.503750,87.350464,100,100,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,639-02-5506,14672_NW_NM5425-Githurai_Roundabout_OUTH_U900-1,QOARNC1106,NAIROBI WEST,32959,Huawei,3G,-1.203720,36.918900,14672_NW_NM3425-Githurai_Roundabout_OUTH_MGF,...,4227.0,95.807549,97.935661,98.462044,0.166168,4.464143,58.481750,100,100,100.0
4996,639-02-4523,14698_NW_NM5426-Zimmerman_Njathaini_OUTH_U900-1,QOARNC0512,NAIROBI WEST,32968,Huawei,3G,-1.204560,36.896198,14698_NW_NM3426-Zimmerman_Njathaini_OUTH_MGF,...,NaN,98.173683,96.613144,98.824318,0.271924,2.154833,72.972717,100,100,100.0
4997,639-02-5571,14223_WN_WL5427-Koru_MGF_UMTS900-4,CHRRNC2501,GREATER WESTERN,32986,Nokia,3G,-0.181306,35.269699,14223_WN_WL3427-Koru_MGF,...,NaN,99.009644,98.796722,99.114143,0.263736,2.188611,84.565735,100,100,100.0
4998,639-02-2530,15582_CO_EL3507-Dabasso_Dongo_Kundu_MGF-0,MSARNC3004,COAST,12874,Nokia,3G,-3.379890,39.980099,15582_CO_EL3507-Dabasso_Dongo_Kundu_MGF,...,NaN,98.384926,98.235291,99.451752,0.053349,2.210723,98.070068,100,100,100.0


In [25]:
#Define a function to calculate Call setup time NEI
def NEI(data):
    for i, row in data.iterrows():
        if data['setup_time'][i]<=4:
            data['Setup time NEI']=100
        elif data['setup_time'][i]<=6:
            data['Setup time NEI']=80 + (6-data['setup_time'])*(100-80)/(6-4)
        elif data['setup_time'][i]<=10:
            data['Setup time NEI']=0 +(10-data['setup_time'])*(80-0)/(10-6)
        else:
            data['Setup time NEI']=0
    return(data)


In [26]:
NEI(cells_df)

,LAC,Cell_Name,Node_Name,Region,ID,Vendor,TECHNOLOGY,LATITUDE_DEC,LONGITUDE_DEC,Site_Name,...,Data_SR_Daily,Data_SR_BH,CEM_CSSR,CEM_CDR,setup_time,NEI CSSR,Data SR NEI,CDR NEI,Availability NEI,Setup time NEI
0,639-02-4526,13865_NE_NI2370-Kibera_South_OUTE-0,ATHRNC1501,NAIROBI EAST,4024,Huawei,3G,-1.318720,36.783600,13865_NE_NI2370-Kibera_South_OUTE_MGF,...,99.333153,100.000000,99.618324,0.063857,0.612605,104.732971,100,100,100.0,133.873947
1,639-02-4526,13865_NE_NI2370-Kibera_South_OUTE-1,ATHRNC1501,NAIROBI EAST,4026,Huawei,3G,-1.318720,36.783600,13865_NE_NI2370-Kibera_South_OUTE_MGF,...,98.729019,98.722740,98.604500,0.582751,2.834374,64.179993,100,100,100.0,111.656260
2,639-02-2518,13814_MK_EC2370-Meru_Tuntu_OUTA-2,NYRRNC4004,MT.KENYA,4027,Huawei,3G,0.054487,37.642799,13814_MK_EC2370-Meru_Tuntu_OUTA_MGF,...,99.939789,100.000000,99.190575,0.114531,4.352467,87.622986,100,100,100.0,96.475334
3,639-02-4510,13605_NE_NI2379-Kayole_Section_1_OUTE-5,QOARNC1100,NAIROBI EAST,4111,Huawei,3G,-1.265400,36.917500,13605_NE_NI2379-Kayole_Section_1_OUTE_MRT,...,99.587761,99.388428,99.054306,0.246063,2.737468,82.172241,100,100,100.0,112.625322
4,639-02-4521,13760_NE_NI2385-Nacico_OUTE_MWM-6,SCCRNC1001,NAIROBI EAST,4175,Huawei,3G,-1.287060,36.837200,13760_NE_NI2385-Nacico_OUTE_MWM,...,99.303093,98.305084,99.183762,0.232116,5.503750,87.350464,100,100,100.0,84.962497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,639-02-5506,14672_NW_NM5425-Githurai_Roundabout_OUTH_U900-1,QOARNC1106,NAIROBI WEST,32959,Huawei,3G,-1.203720,36.918900,14672_NW_NM3425-Githurai_Roundabout_OUTH_MGF,...,95.807549,97.935661,98.462044,0.166168,4.464143,58.481750,100,100,100.0,95.358567
4996,639-02-4523,14698_NW_NM5426-Zimmerman_Njathaini_OUTH_U900-1,QOARNC0512,NAIROBI WEST,32968,Huawei,3G,-1.204560,36.896198,14698_NW_NM3426-Zimmerman_Njathaini_OUTH_MGF,...,98.173683,96.613144,98.824318,0.271924,2.154833,72.972717,100,100,100.0,118.451667
4997,639-02-5571,14223_WN_WL5427-Koru_MGF_UMTS900-4,CHRRNC2501,GREATER WESTERN,32986,Nokia,3G,-0.181306,35.269699,14223_WN_WL3427-Koru_MGF,...,99.009644,98.796722,99.114143,0.263736,2.188611,84.565735,100,100,100.0,118.113894
4998,639-02-2530,15582_CO_EL3507-Dabasso_Dongo_Kundu_MGF-0,MSARNC3004,COAST,12874,Nokia,3G,-3.379890,39.980099,15582_CO_EL3507-Dabasso_Dongo_Kundu_MGF,...,98.384926,98.235291,99.451752,0.053349,2.210723,98.070068,100,100,100.0,117.892766


In [27]:
cells_df['NEI']=(cells_df['NEI CSSR']+cells_df['Data SR NEI']+cells_df['CDR NEI']+cells_df['Availability NEI']+cells_df['Setup time NEI'])/5

In [28]:
cells_df.columns

Index(['LAC', 'Cell_Name', 'Node_Name', 'Region', 'ID', 'Vendor', 'TECHNOLOGY',
       'LATITUDE_DEC', 'LONGITUDE_DEC', 'Site_Name', 'BAND', 'Azimuth',
       'distname_cell_name', 'Sales_Area', 'Sales_Zone', 'Field_Engineer',
       'Optimization_Engineer', 'Cell_Classification', 'RAN_did', 'Date',
       'did', 'CS_Traffic_BH', 'DL_Volume_BH', 'UL_Volume_BH', 'TCH_Drops_BH',
       'TCH_Assignments_BH', 'Status', 'CSSR_BH', 'SD_Seizure_Success_BH_N1',
       'SD_Seizure_Attemps_BH_D1', 'SD_Success_BH_N2',
       'SD_Assign_Attempts_BH_D2', 'TCH_Assignments_Success_BH_N3',
       'TCH_Assignments_Success_BH_D3', 'TBF_Success_BH', 'TBF_Attempts_BH',
       'CS_RRC_Success_BH_N1', 'CS_RRC_Attempts_BH_D1', 'CS_RAB_Success_BH_N2',
       'CS_RAB_Attempts_BH_D2', 'PS_RAB_Success_BH_N1',
       'PS_RAB_Attempts_BH_D1', 'PS_RRC_Success_BH_N2',
       'PS_RRC_Attempts_BH_D2', 'Succesfull TCH Sezieures', 'CS_TRAffic_Daily',
       'TCH_Availability_Daily', 'DL_Volume_Daily', 'UL_Volume_Daily',

if Regional['CEM_CSSR'].any()>99:
    Regional['NEI CSSR']=(80 + ((Regional['CEM_CSSR'].any()-99)*(100-80)/(99.5-99))) 

In [29]:
#Regional['NEI CSSR'] =Regional['CEM_CSSR'].apply(lambda x: 80 + (Regional['CEM_CSSR']-99)*(100-80)/(99.5-99) if x>99 else 0 +(Regional['CEM_CSSR']-85)*(80-0)/(99-85) if x>85 else 0)
#Regional['NEI CSSR'] =Regional['CEM_CSSR'].apply(lambda x: 'A' if x>99 else 'B' if x>85 else 'C')
#Regional['Data_SR_BH'] =Regional['Data_SR_BH'].apply(lambda x: (80 + ((Regional['Data_SR_BH']-99)*(100-80)/(99.5-99)))if x>99 else (0 +((Regional['Data_SR_BH']-85)*(80-0)/(99-85))) if x>85 else 0)

In [30]:
cells_df1=cells_df[['Region','Sales_Area','Node_Name','Site_Name','Cell_Name','LATITUDE_DEC','LONGITUDE_DEC','NEI','NEI CSSR','Data SR NEI', 'CDR NEI', 'Availability NEI', 'Setup time NEI']]

In [31]:
cells_dfx=cells_df1.groupby(['Region']).mean()

In [32]:
cells_dfx

,LATITUDE_DEC,LONGITUDE_DEC,NEI,NEI CSSR,Data SR NEI,CDR NEI,Availability NEI,Setup time NEI
Region,,,,,,,,
COAST,-3.755645,39.663482,79.865882,81.538913,100,100,5.858912,109.645099
GREATER WESTERN,-0.008700,34.634890,87.713145,82.488126,100,100,49.429173,108.922975
MT.KENYA,-0.129434,37.654018,85.611880,82.202546,100,100,39.425990,109.703244
NAIROBI EAST,-1.431205,37.054138,84.450072,83.168832,100,100,-0.130368,110.996252
NAIROBI WEST,-1.252271,36.825803,88.414195,83.117688,100,100,11.494342,109.757044
RIFT,0.137835,35.612546,79.999179,82.554084,100,100,-8.756722,109.376215


In [33]:
gpd_regions=gpd_regions.rename(columns={'REGION':'Region'})
Regional=pd.merge(cells_dfx,gpd_regions[['Region','geometry']], on=['Region'])
Regional.shape

(6, 10)

In [34]:
gpd_regional = gpd.GeoDataFrame(Regional, geometry=Regional['geometry'])
gpd_regional.crs = {'init':'epsg:4326'}

In [35]:
Region_map.add_data(data=gpd_regional, name="Saf_Regions")

In [36]:
Region_map

KeplerGl(data={'Saf_Regions': {'index': [0, 1, 2, 3, 4, 5], 'columns': ['Region', 'LATITUDE_DEC', 'LONGITUDE_D…

## Sales Areas

In [37]:
cells_dfxx=cells_df1.groupby(['Sales_Area']).mean()

In [39]:
gpd_Sales_Area=gpd_Sales_Area.rename(columns={'SALESAREA':'Sales_Area'})
Sales_Area=pd.merge(cells_dfxx,gpd_Sales_Area[['Sales_Area','geometry']], on=['Sales_Area'])
Sales_Area.shape

(41, 10)

In [40]:
gpd_Sales_Area = gpd.GeoDataFrame(Sales_Area, geometry=Sales_Area['geometry'])
gpd_Sales_Area.crs = {'init':'epsg:4326'}

In [41]:
Region_map.add_data(data=gpd_Sales_Area, name="Saf_Sales_Area")

In [42]:
Region_map

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'u5vmzg', 'type': '…

## TDR Regions

In [43]:
#Since TDR regions are not defined on the DB we do a spatial join between site coordinates and the TDR gpd data

In [44]:
cells_df1.head()

,Region,Sales_Area,Node_Name,Site_Name,Cell_Name,LATITUDE_DEC,LONGITUDE_DEC,NEI,NEI CSSR,Data SR NEI,CDR NEI,Availability NEI,Setup time NEI
0,NAIROBI EAST,NAIROBI SOUTH,ATHRNC1501,13865_NE_NI2370-Kibera_South_OUTE_MGF,13865_NE_NI2370-Kibera_South_OUTE-0,-1.318720,36.783600,107.721384,104.732971,100,100,100.0,133.873947
1,NAIROBI EAST,NAIROBI SOUTH,ATHRNC1501,13865_NE_NI2370-Kibera_South_OUTE_MGF,13865_NE_NI2370-Kibera_South_OUTE-1,-1.318720,36.783600,95.167251,64.179993,100,100,100.0,111.656260
2,MT.KENYA,UPPER MOUNTAIN,NYRRNC4004,13814_MK_EC2370-Meru_Tuntu_OUTA_MGF,13814_MK_EC2370-Meru_Tuntu_OUTA-2,0.054487,37.642799,96.819664,87.622986,100,100,100.0,96.475334
3,NAIROBI EAST,KANGUNDO ROAD,QOARNC1100,13605_NE_NI2379-Kayole_Section_1_OUTE_MRT,13605_NE_NI2379-Kayole_Section_1_OUTE-5,-1.265400,36.917500,98.959513,82.172241,100,100,100.0,112.625322
4,NAIROBI EAST,EASTLANDS VIWANDANI,SCCRNC1001,13760_NE_NI2385-Nacico_OUTE_MWM,13760_NE_NI2385-Nacico_OUTE_MWM-6,-1.287060,36.837200,94.462592,87.350464,100,100,100.0,84.962497


In [45]:
#cells_df1['Long-Lat'] = list(zip(cells_df1.LONGITUDE_DEC, cells_df1.LATITUDE_DEC))
#cells_df1['Long-Lat'] = cells_df1['Long-Lat'].apply(Point)

In [46]:
x= gpd.GeoDataFrame(cells_df1, geometry=gpd.points_from_xy(cells_df1.LONGITUDE_DEC, cells_df1.LATITUDE_DEC))
x.crs = {'init':'epsg:4326'}

In [47]:
x.head()

,Region,Sales_Area,Node_Name,Site_Name,Cell_Name,LATITUDE_DEC,LONGITUDE_DEC,NEI,NEI CSSR,Data SR NEI,CDR NEI,Availability NEI,Setup time NEI,geometry
0,NAIROBI EAST,NAIROBI SOUTH,ATHRNC1501,13865_NE_NI2370-Kibera_South_OUTE_MGF,13865_NE_NI2370-Kibera_South_OUTE-0,-1.318720,36.783600,107.721384,104.732971,100,100,100.0,133.873947,POINT (36.78360 -1.31872)
1,NAIROBI EAST,NAIROBI SOUTH,ATHRNC1501,13865_NE_NI2370-Kibera_South_OUTE_MGF,13865_NE_NI2370-Kibera_South_OUTE-1,-1.318720,36.783600,95.167251,64.179993,100,100,100.0,111.656260,POINT (36.78360 -1.31872)
2,MT.KENYA,UPPER MOUNTAIN,NYRRNC4004,13814_MK_EC2370-Meru_Tuntu_OUTA_MGF,13814_MK_EC2370-Meru_Tuntu_OUTA-2,0.054487,37.642799,96.819664,87.622986,100,100,100.0,96.475334,POINT (37.64280 0.05449)
3,NAIROBI EAST,KANGUNDO ROAD,QOARNC1100,13605_NE_NI2379-Kayole_Section_1_OUTE_MRT,13605_NE_NI2379-Kayole_Section_1_OUTE-5,-1.265400,36.917500,98.959513,82.172241,100,100,100.0,112.625322,POINT (36.91750 -1.26540)
4,NAIROBI EAST,EASTLANDS VIWANDANI,SCCRNC1001,13760_NE_NI2385-Nacico_OUTE_MWM,13760_NE_NI2385-Nacico_OUTE_MWM-6,-1.287060,36.837200,94.462592,87.350464,100,100,100.0,84.962497,POINT (36.83720 -1.28706)


In [48]:
gpd_TDR.head()

,REGION,SALES_AREA,SUBREGION,TDR_NAME,FINAL_TDR,POP,AREA,geometry
0,COAST,NORTH COAST,MOMBASA,None,SHANZU,13380.513936,0.0,"POLYGON ((39.69933 -4.00192, 39.70050 -4.00003..."
1,COAST,MALINDI,MOMBASA,None,MALINDI,27751.462723,0.0,"POLYGON ((40.07912 -3.26399, 40.07954 -3.26124..."
2,COAST,TSAVO,MOMBASA,None,MAZERAS,86139.134392,0.0,"MULTIPOLYGON (((39.45101 -3.98370, 39.45112 -3..."
3,COAST,NORTH COAST,MOMBASA,None,MTWAPA,14472.473233,0.0,"POLYGON ((39.67649 -3.91797, 39.68172 -3.89609..."
4,COAST,TSAVO,MOMBASA,None,KALOLENI,132432.316754,0.0,"POLYGON ((39.54339 -3.62387, 39.56654 -3.63220..."


In [49]:
TDR_regions = gpd.sjoin( x,gpd_TDR, op='within')

C:\Users\mnmureithi\AppData\Local\Continuum\anaconda3\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!({'init': 'epsg:4326'} != {'init': 'epsg:4326', 'no_defs': True})
  "(%s != %s)" % (left_df.crs, right_df.crs)
C:\Users\mnmureithi\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [50]:
TDR_regions['FINAL_TDR'].nunique()

301

In [51]:
TDR_regions.head()

,Region,Sales_Area,Node_Name,Site_Name,Cell_Name,LATITUDE_DEC,LONGITUDE_DEC,NEI,NEI CSSR,Data SR NEI,...,Setup time NEI,geometry,index_right,REGION,SALES_AREA,SUBREGION,TDR_NAME,FINAL_TDR,POP,AREA
0,NAIROBI EAST,NAIROBI SOUTH,ATHRNC1501,13865_NE_NI2370-Kibera_South_OUTE_MGF,13865_NE_NI2370-Kibera_South_OUTE-0,-1.31872,36.783600,107.721384,104.732971,100,...,133.873947,POINT (36.78360 -1.31872),268,NAIROBI EAST,NAIROBI SOUTH,NAIROBI SOUTH,None,KARENGATA,0.0,0.0
1,NAIROBI EAST,NAIROBI SOUTH,ATHRNC1501,13865_NE_NI2370-Kibera_South_OUTE_MGF,13865_NE_NI2370-Kibera_South_OUTE-1,-1.31872,36.783600,95.167251,64.179993,100,...,111.656260,POINT (36.78360 -1.31872),268,NAIROBI EAST,NAIROBI SOUTH,NAIROBI SOUTH,None,KARENGATA,0.0,0.0
47,NAIROBI EAST,NAIROBI SOUTH,ATHRNC1501,14251_NE_NI2369-Southland_Landless_OUTE_MGF,14251_NE_NI2369-Southland_Landless_OUTE-9,-1.32211,36.795898,104.937496,90.809326,100,...,133.878155,POINT (36.79590 -1.32211),268,NAIROBI EAST,NAIROBI SOUTH,NAIROBI SOUTH,None,KARENGATA,0.0,0.0
55,NAIROBI EAST,NAIROBI SOUTH,ATHRNC1501,13865_NE_NI2370-Kibera_South_OUTE_MGF,13865_NE_NI2370-Kibera_South_OUTE-6,-1.31872,36.783600,95.971298,89.230652,100,...,129.206163,POINT (36.78360 -1.31872),268,NAIROBI EAST,NAIROBI SOUTH,NAIROBI SOUTH,None,KARENGATA,0.0,0.0
88,NAIROBI EAST,NAIROBI SOUTH,ATHRNC1501,12002_NE_NI2003-Onyonka_OUTE_MGF,12002_NE_NI2003-Onyonka_OUTE-6,-1.32622,36.787998,105.223943,92.554321,100,...,133.565396,POINT (36.78800 -1.32622),268,NAIROBI EAST,NAIROBI SOUTH,NAIROBI SOUTH,None,KARENGATA,0.0,0.0


In [52]:
TDR_regions1=TDR_regions.groupby(['FINAL_TDR']).mean()
TDR_regions1.reset_index(inplace=True)

In [53]:
TDR_regions1.head()

,FINAL_TDR,LATITUDE_DEC,LONGITUDE_DEC,NEI,NEI CSSR,Data SR NEI,CDR NEI,Availability NEI,Setup time NEI,index_right,POP,AREA
0,2ND AVENUE,-1.267392,36.847457,99.373769,90.607522,100,100,93.516188,113.924011,28,35996.296877,0.0
1,AA NJENGA,-1.321725,36.886726,99.963090,90.434723,100,100,100.000000,109.380729,283,0.000000,0.0
2,ACCRA ROAD,-1.282923,36.826472,99.400795,83.468366,100,100,100.000000,113.535610,126,2720.064931,0.0
3,AHERO,-0.276482,34.941868,92.783709,88.785428,100,100,37.428523,114.668854,194,307207.500087,0.0
4,AMBASADOR,-1.286242,36.827293,55.315473,84.468942,100,100,-119.486947,111.595370,123,11767.553419,0.0


In [54]:
gpd_TDR.head()

,REGION,SALES_AREA,SUBREGION,TDR_NAME,FINAL_TDR,POP,AREA,geometry
0,COAST,NORTH COAST,MOMBASA,None,SHANZU,13380.513936,0.0,"POLYGON ((39.69933 -4.00192, 39.70050 -4.00003..."
1,COAST,MALINDI,MOMBASA,None,MALINDI,27751.462723,0.0,"POLYGON ((40.07912 -3.26399, 40.07954 -3.26124..."
2,COAST,TSAVO,MOMBASA,None,MAZERAS,86139.134392,0.0,"MULTIPOLYGON (((39.45101 -3.98370, 39.45112 -3..."
3,COAST,NORTH COAST,MOMBASA,None,MTWAPA,14472.473233,0.0,"POLYGON ((39.67649 -3.91797, 39.68172 -3.89609..."
4,COAST,TSAVO,MOMBASA,None,KALOLENI,132432.316754,0.0,"POLYGON ((39.54339 -3.62387, 39.56654 -3.63220..."


In [55]:
#gpd_TDR_data=gpd_Sales_Area.rename(columns={'SALESAREA':'Sales_Area'})
gpd_TDR_data=pd.merge(TDR_regions1,gpd_TDR[['FINAL_TDR','geometry']], on=['FINAL_TDR'])
gpd_TDR_data.shape

(301, 13)

In [56]:
gpd_TDR_data.head()

,FINAL_TDR,LATITUDE_DEC,LONGITUDE_DEC,NEI,NEI CSSR,Data SR NEI,CDR NEI,Availability NEI,Setup time NEI,index_right,POP,AREA,geometry
0,2ND AVENUE,-1.267392,36.847457,99.373769,90.607522,100,100,93.516188,113.924011,28,35996.296877,0.0,"POLYGON ((36.84588 -1.26115, 36.84582 -1.26058..."
1,AA NJENGA,-1.321725,36.886726,99.963090,90.434723,100,100,100.000000,109.380729,283,0.000000,0.0,"POLYGON ((36.89607 -1.32627, 36.89223 -1.32815..."
2,ACCRA ROAD,-1.282923,36.826472,99.400795,83.468366,100,100,100.000000,113.535610,126,2720.064931,0.0,"POLYGON ((36.82804 -1.28149, 36.82801 -1.28197..."
3,AHERO,-0.276482,34.941868,92.783709,88.785428,100,100,37.428523,114.668854,194,307207.500087,0.0,"POLYGON ((35.10836 -0.33609, 35.10818 -0.33627..."
4,AMBASADOR,-1.286242,36.827293,55.315473,84.468942,100,100,-119.486947,111.595370,123,11767.553419,0.0,"POLYGON ((36.82542 -1.28510, 36.82548 -1.28501..."


In [57]:
#Converting to a GeoDataFrame
gpd_TDR_data = gpd.GeoDataFrame(Sales_Area, geometry=gpd_TDR_data['geometry'])
gpd_TDR_data.crs = {'init':'epsg:4326'}

In [58]:
Region_map.add_data(data=gpd_TDR_data, name="Saf_TDR_Zones")

In [59]:
Region_map

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'u5vmzg', 'type': '…